In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os

In [2]:
import cv2
from glob import glob

train_images = [] 
train_labels = []


In [3]:
def load_images(classed):
    # Loop over the training folder 
    
    print('Loading directory c{}'.format(classed))
    files = glob(os.path.join('..', 'input','state-farm-distracted-driver-detection','imgs', 'train', 'c' + str(classed), '*.jpg'))
    for file in files:
        img = cv2.imread(file)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY) 
        train_images.append(cv2.resize(img, (225,225)))
        train_labels.append(classed)


In [4]:
load_images(0)
load_images(2)
load_images(4)
load_images(6)

Loading directory c0
Loading directory c2
Loading directory c4
Loading directory c6


In [5]:
from sklearn.model_selection import train_test_split
import tensorflow as tf

x_train, x_test, y_train, y_test =  train_test_split(train_images,train_labels, test_size = 0.25)

ModuleNotFoundError: No module named 'tensorflow'

In [ ]:

y_train = tf.keras.utils.to_categorical(y_train, num_classes=10)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=10)

x_train, y_train = np.asarray(x_train), np.asarray(y_train)
x_test, y_test = np.asarray(x_test), np.asarray(y_test)
print(x_train.shape, y_train.shape)

In [ ]:
x_test = np.expand_dims(x_test, axis=3)
x_train = np.expand_dims(x_train, axis=3)

# Exploring the dataset

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(cv2.cvtColor(x_train[1000], cv2.COLOR_BGR2RGB))
plt.show()
print(y_train[1000])

# CNN Model

In [ ]:
from keras.utils import to_categorical
from keras.models import Sequential
from keras.layers import *
from keras.optimizers import RMSprop
import keras
from keras.callbacks import EarlyStopping


model = Sequential()

size = 16

model.add(Conv2D(size*2, (3, 3), padding='same', input_shape=(225,225,1)))
model.add(Activation('elu'))
model.add(Conv2D(size*2, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(size*4, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(size*4, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))

model.add(Conv2D(size*8, (3, 3), padding='same'))
model.add(Activation('elu'))
model.add(Conv2D(size*8, (3, 3)))
model.add(Activation('elu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))


model.add(Flatten())
model.add(Dense(size*16))
model.add(Activation('elu'))
model.add(Dropout(0.5))
model.add(Dense(10))
model.add(Activation('softmax'))

# initiate RMSprop optimizer
opt = keras.optimizers.RMSprop(lr=0.0001, decay=1e-6)

# Let's train the model using RMSprop
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])


In [ ]:
model.summary()

In [ ]:
history = model.fit(x_train, y_train, 
          validation_data=(x_test, y_test),
          epochs=8, batch_size=25, callbacks = EarlyStopping(monitor='val_loss', mode='min', patience=2))

In [ ]:
plt.figure(figsize=(8,8))
plt.subplot(211)
plt.title('Cross Entropy Loss')
plt.plot(history.history['loss'], color='blue', label='train')
plt.plot(history.history['val_loss'], color='orange', label='test')
# plot accuracy
plt.subplot(212)
plt.title('Classification Accuracy')
plt.plot(history.history['accuracy'], color='blue', label='train')
plt.plot(history.history['val_accuracy'], color='orange', label='test')

In [ ]:
from keras.models import load_model

model.save('distracted_drivers.h5')

In [ ]:
_, acc = model.evaluate(x_test, y_test, verbose=10)
print(acc)